### 0.0.2 Monte Carlo Simulation

In [ ]:
import optuna
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from prettytable import PrettyTable
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error


class CustomAlgorithm:
    #insert custom algorithm

def optimize_and_evaluate(X_, y_, X_test, y_test, use_customalgorithm=True):
    X_train, X_valid, y_train, y_valid = train_test_split(X_, y_, test_size=0.3, random_state=42)
    
    def objective(trial):
        params_trial = {
            'n_estimators': trial.suggest_categorical('n_estimators', [100, 150, 200, 300]),
            'max_depth': trial.suggest_categorical('max_depth', [None, 5, 10, 20, 30]),
            'min_samples_split': trial.suggest_categorical('min_samples_split', [2, 5, 10]),
            'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [1, 2, 4]),
            'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
            'random_state': 42
        }
        estimator = RandomForestRegressor(**params_trial)
        
        if use_customalgorithm:
            custom = trial.suggest_categorical('k', [3, 4, 5, 6])
            model = CustomAlgorithm(estimator)
        else:
            model = estimator
            
        model.fit(X_train, y_train)
        yhat_valid = model.predict(X_valid)
        mse = mean_squared_error(y_valid, yhat_valid)
        return mse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)
    best_params = study.best_params
    best_mse = study.best_value

    if use_customalgorithm:
        custom = best_params.pop()
        estimator = RandomForestRegressor(**best_params)
        model = CustomAlgorithm(estimator)
    else:
        estimator = RandomForestRegressor(**best_params)
        model = estimator

    model.fit(X_train, y_train)
    
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)


    #The main goal of the optuna study is to get the result with the lowest MSE, there might be other results with better SMAPE/MAE
    #but I am choosing to work with MSE here
    smape = 100 * np.mean(2 * np.abs(yhat_test - y_test) / (np.abs(yhat_test) + np.abs(y_test)))
    mae = mean_absolute_error(y_test, yhat_test)
    mse = mean_squared_error(y_test, yhat_test)

    return best_params, best_mse, smape, mae, mse

# Function to run the Monte Carlo simulation
def monte_carlo_simulation(n_iterations, n_samples=100):
    results = {'CustomAlgorithm': {'MAE': [], 'MSE': [], 'SMAPE': []}, 'Regressor': {'MAE': [], 'MSE': [], 'SMAPE': []}}

    for i in range(n_iterations):
        X, y = make_regression(n_samples=n_samples, n_features=5, noise=50, random_state=i)

        # Adding interaction terms to generate some variance in the dataset
        interaction_terms = X[:, :5] * X[:, :5]  # Adjust the range to cover all features
        X = np.hstack([X, interaction_terms])

        # Modifying noise and introduce outliers for the same reason as above
        np.random.seed(i)
        noise = np.random.normal(0, 20, size=y.shape)
        y += noise
        outliers = np.random.choice(np.arange(len(y)), size=5, replace=False)
        y[outliers] *= 3 

        # Applying shifting variations (which was the main thing I did in the first dataset I made)
        # and it seems like a good idea to change the amount of points and the shift values to introduce variance
        points_to_shift = random.randint(3, 8)                     
        shift_values = np.arange(1, 2.0, 0.25)
        shift = random.choice(shift_values)

        min_indices_y = np.argpartition(y, points_to_shift)[:points_to_shift]
        max_indices_y = np.argpartition(y, -points_to_shift)[-points_to_shift:]

        X[min_indices_y] -= shift
        y[min_indices_y] -= shift
        X[max_indices_y] += shift
        y[max_indices_y] += shift

        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

        _, _, smape_cal, mae_cal, mse_cal = optimize_and_evaluate(X_train, y_train, X_test, y_test, use_customalgorithm=True)
        _, _, smape_rf, mae_rf, mse_rf = optimize_and_evaluate(X_train, y_train, X_test, y_test, use_customalgorithm=False)

        # Store results
        results['CustomAlgorithm']['MAE'].append(mae_cal)
        results['CustomAlgorithm']['MSE'].append(mse_cal)
        results['CustomAlgorithm']['SMAPE'].append(smape_cal)

        results['Regressor']['MAE'].append(mae_rf)
        results['Regressor']['MSE'].append(mse_rf)
        results['Regressor']['SMAPE'].append(smape_rf)

    return results


# Run Monte Carlo simulation with 10 iterations
n_iterations = 30
results = monte_carlo_simulation(n_iterations)


# Store the means for the comparison
mean_mae_cal = np.mean(results['CustomAlgorithm']['MAE'])
mean_mse_cal= np.mean(results['CustomAlgorithm']['MSE'])
mean_smape_cal = np.mean(results['CustomAlgorithm']['SMAPE'])

mean_mae_rf = np.mean(results['Regressor']['MAE'])
mean_mse_rf = np.mean(results['Regressor']['MSE'])
mean_smape_rf = np.mean(results['Regressor']['SMAPE'])


# Display results
print("CustomAlgorithm Results:")
print(f"Mean MAE: {np.mean(results['CustomAlgorithm']['MAE'])}")
print(f"Mean MSE: {np.mean(results['CustomAlgorithm']['MSE'])}")
print(f"Mean SMAPE: {np.mean(results['CustomAlgorithm']['SMAPE'])}")

print("Base_Regressor Results:")
print(f"Mean MAE: {np.mean(results['Regressor']['MAE'])}")
print(f"Mean MSE: {np.mean(results['Regressor']['MSE'])}")
print(f"Mean SMAPE: {np.mean(results['Regressor']['SMAPE'])}")

### 0.0.3 Histograms

In [ ]:
# Plot histograms combined
metrics = ['MAE', 'MSE', 'SMAPE']
for metric in metrics:
    plt.figure(figsize=(10, 6))
    
    # Histogram for both Regressor and CustomAlgorithm
    plt.hist(results['CustomAlgorithm'][metric], bins=20, alpha=0.7, color='blue', label='CustomAlgorithm', histtype = "barstacked")
    plt.hist(results['Regressor'][metric], bins=20, alpha=0.5, color='orange', label='Base_Regressor', histtype = "barstacked")
    
    plt.title(f'{metric} Histogram')
    plt.xlabel(metric)
    plt.ylabel('Frequency')
    plt.legend(loc='upper right')
    
    plt.show()

### 0.0.4 Comparison

In [ ]:
def compare_metrics(metric_cal, metric_rf, metric_name):
    print('CustomAlgorithm score: %g | base Regressor score: %g' % (metric_cal, metric_rf))

    if metric_cal < metric_rf:
        print(f"The model is more accurate with CustomAlgorithm in terms of {metric_name}.")
        return

    elif metric_cal == metric_rf:
        print("Further examination is required to determine superiority.")
        return

    else:
        print(f"The CustomAlgorithm model is not better in terms of {metric_name}.")
        return
        

compare_metrics(mean_smape_cal, mean_smape_rf, "SMAPE")
compare_metrics(mean_mae_cal, mean_mae_rf, "MAE")
compare_metrics(mean_mse_cal, mean_mse_rf, "MSE")